In [22]:
from Geometry import *
import numpy as np

def getAllDecendants(octTree):
    '''Get all lowest chilren on tree.'''
    out = []
    if octTree.hasChildren:
        for child in octTree.children:
            out = out + getAllDecendants(child)
        return out
    else:
        return [octTree]
    
def generateModelFromOctree(octTree,numRays,propName='Ne'):
    '''Generate '''
    voxels = getAllDecendants(octTree)        
    N = len(voxels)
    G = np.zeros([numRays,N])
    m = np.zeros(N)
    Cm = np.zeros(N)
    x = np.zeros([N,3])
    if propName not in voxels[0].properties.keys():
        #zero model if no property
        i = 0
        while i < N:
            vox = voxels[i]
            for j in vox.lineSegments.keys():
                G[j,i] = vox.lineSegments[j].sep   
            x[i,:] = vox.centroid
            i += 1
        return G,Cm,m,x
    i = 0
    while i < N:
        vox = voxels[i]
        for j in vox.lineSegments.keys():
            G[j,i] = vox.lineSegments[j].sep   
        m[i] = vox.properties[propName][1]
        Cm[i] = vox.properties[propName][2]
        x[i,:] = vox.centroid
        i += 1
    return G,Cm,m,x
        
def setOctTreeModel(octTree,m,Cm,propName='Ne',propType='extensive'):
    '''Set the model in the octTree. 
    Assumes the model is derived from the same octTree and
    Cm is the diagonal of the covariance.'''
    voxels = getAllDecendants(octTree)
    N = len(voxels)
    i = 0
    while i < N:
        vox = voxels[i]
        vox.properties[propName] = [propType,m[i],Cm[i]]
        vox.lineSegments = {}
        i += 1
    
def forwardProblem(rays,octTree):
    '''Perform the forward problem.'''
    
    pass
    
def noRefractionTECInversion(antennaLocs,antennaTEC, sourcePos):
    '''Invert the 3d TEC based on total measurments on the ground (after factor)
    antennaLocs are in uvw coords of shape
    antennaTEC are NxM'''
    pass

def makeRaysFromSourceAndReciever(recievers,sources):
    rays = []
    count = 0
    for r in recievers:
        for s in sources:
            rays.append(LineSegment(r,s,id=count))
            count += 1
    return rays

def LinearSolution(dobs,G,Cd,Cmprior,mprior):
    #forward problem
    print("Doing forward problem")
    d = G.dot(mprior)
    print("Calculating residuals:")
    residuals = dobs - d
    print(residuals)
    Gt = G.transpose()
    #smooth and adjoint
    print("Calculating smoothing matrix")
    smooth = np.linalg.inv(G.dot(Cmprior).dot(Gt) + Cd)
    print("Calculating adjoint")
    adjoint = Cmprior.dot(Gt).dot(smooth)
    print("updating model")
    m = mprior + adjoint.dot(residuals)
    print("updating covariance")
    Cm = Cmprior - adjoint.dot(G).dot(Cmprior)
    return m,Cm  



    
def compute3dExponentialCovariance(sigma2,L,x):
    N = x.shape[0]
    C = np.zeros([N,N])
    i = 0
    while i < N:
        C[i,i] = sigma2
        j = i+1
        while j < N:
            d = x[i,:] - x[j,:]
            C[i,j] = sigma2*np.exp(-np.linalg.norm(d)/L)
            C[j,i] = C[i,j]
            j += 1
        i += 1
    C[C<epsFloat] = 0.
    return C

def ExampleLinearSolution():
    levels = 3
    fileName = "octTree_{0}levels.npy".format(levels)
    baseWidth = 1.
    height = 1000.
    try:
        octTree = loadOctTree(fileName)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    except:
        octTree = OctTree([0,0,height/2.],dx=baseWidth,dy=baseWidth,dz=height)
        subDivideToDepth(octTree,levels)
        saveOctTree(fileName,octTree)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
        
    
    mexact = np.zeros_like(m)
    M = 2
    print("Generating {0} component model".format(M))
    for i in range(M):
        loc = np.array([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = height/2.,high = height)])
        mexact += 0.3 + np.random.uniform()*np.exp(-np.sum((x - loc)**2,axis=1)/(height/8.)**2)
    modelVar = 0.1
    print("Computing model 3d exponential covariance")
    Cmexact = compute3dExponentialCovariance(modelVar,baseWidth/2.,x)
    print("Setting octTree with model.")
    setOctTreeModel(octTree,mexact,np.diag(Cmexact),propName='Ne',propType='extensive')
    M = 5
    print("Generating {0} recievers".format(M))
    recievers = []
    for i in range(M):
        recievers.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               -epsFloat])
    M = 10
    print("Generating {0} sources".format(M))
    sources = []
    for i in range(M):
        sources.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               height])
    rays = makeRaysFromSourceAndReciever(recievers,sources)
    print("Propagating {0} rays".format(len(rays)))
    for ray in rays:
        forwardRay(ray,octTree)
    print("Pulling ray propagations.")
    G,CmVar,mexact,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
    
    #generate model
    print("Computing fake observations")
    dobs = []
    for i in range(20):
        dobs.append(G.dot(np.random.multivariate_normal(mean=mexact, cov = Cmexact)))
        #print(dobs[-1])
    dobs = np.array(dobs)
    Cd = np.cov(dobs.transpose())
    dobs = np.mean(dobs,axis=0)
    #print (dobs)
    mprior = mexact/0.5
    print("Solving for model from rays:")
    print("solution with Cm exact input")
    m,Cm = LinearSolution(dobs,G,Cd,Cmexact,mprior)
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact,label='ex')
    plt.plot(mprior,label='pri')
    plt.legend(frameon=False)
    plt.show()
    plotOctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m)
    f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
    ax1.imshow(Cm)
    ax2.imshow(Cmexact)
    plt.show()
    print("solution with Cm diagonal")
    m,Cm = LinearSolution(dobs,G,Cd,np.diag(np.diag(Cmexact)),mprior)
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact,label='ex')
    plt.plot(mprior,label='pri')
    plt.legend(frameon=False)
    plt.show()
    plotOctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m)
    f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
    ax1.imshow(Cm)
    ax2.imshow(Cmexact)
    plt.show()
    
def SteepestDescent(octTree,rays,dobs,Cd,Cmprior,mprior):
    iter = 0
    mn = mprior
    while iter < 100:
        #forward problem
        print("Setting octTree with model_{0}".format(iter))
        setOctTreeModel(octTree,mn,np.diag(Cmprior),propName='Ne',propType='extensive')
        print("Propagating {0} rays".format(len(rays)))
        for ray in rays:
            forwardRay(ray,octTree)
        print("Pulling ray propagations.")
        G,CmVar,mexact,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
        print("Doing forward problem")
        d = G.dot(mn)
        print("Calculating residuals, Sum:")
        residuals = d - dobs
        print(np.sum(residuals**2))
        print("Calculating weighting residuals")
        weightedRes = np.linalg.pinv(Cd).dot(residuals)
        #print(weightedRes,np.linalg.solve(Cd,residuals))
        #next part should be changed
        Gt = G.transpose()
        #smooth and adjoint
        print("Calculating adjoint")
        dm = Cmprior.dot(Gt).dot(weightedRes)
        print("updating model")
        mn = mn - 0.01*(dm + mn - mprior)
        iter += 1
    
    print("updating covariance")
    print("Calculating smoothing matrix")
    smooth = np.linalg.pinv(G.dot(Cmprior).dot(Gt) + Cd)
    print("Calculating adjoint")
    adjoint = Cmprior.dot(Gt).dot(smooth)
    Cm = Cmprior - adjoint.dot(G).dot(Cmprior)
    return mn,Cm  

def constructIonosphereModel(maxBaseline,):
    levels = 3
    fileName = "ionosphereModel_{0}levels.npy".format(levels)
    baseWidth = 1.
    height = 1000.#km
    try:
        octTree = loadOctTree(fileName)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    except:
        octTree = OctTree([0,0,height/2.],dx=baseWidth,dy=baseWidth,dz=height)
        subDivideToDepth(octTree,levels)
        saveOctTree(fileName,octTree)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    
    
if __name__=='__main__':
    #ExampleLinearSolution()
    levels = 3
    fileName = "octTree_{0}levels.npy".format(levels)
    baseWidth = 1.
    height = 1000.
    try:
        octTree = loadOctTree(fileName)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
    except:
        octTree = OctTree([0,0,height/2.],dx=baseWidth,dy=baseWidth,dz=height)
        subDivideToDepth(octTree,levels)
        saveOctTree(fileName,octTree)
        G,Cm,m,x = generateModelFromOctree(octTree,0,propName='Ne')
        
    
    mexact = np.zeros_like(m)
    M = 3
    print("Generating {0} component model".format(M))
    for i in range(M):
        loc = np.array([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = height/2.,high = height)])
        mexact += 0.3 + np.random.uniform()*np.exp(-np.sum((x - loc)**2,axis=1)/(height/8.)**2)
    modelVar = 0.1
    print("Computing model 3d exponential covariance")
    Cmexact = compute3dExponentialCovariance(modelVar,baseWidth/2.,x)
    print("Setting octTree with model.")
    setOctTreeModel(octTree,mexact,np.diag(Cmexact),propName='Ne',propType='extensive')
    M = 10
    print("Generating {0} recievers".format(M))
    recievers = []
    for i in range(M):
        recievers.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               -epsFloat])
    M = 10
    print("Generating {0} sources".format(M))
    sources = []
    for i in range(M):
        sources.append([np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               np.random.uniform(low = -baseWidth/2.,high = baseWidth/2.),
               height])
    rays = makeRaysFromSourceAndReciever(recievers,sources)
    print("Propagating {0} rays".format(len(rays)))
    for ray in rays:
        forwardRay(ray,octTree)
    print("Pulling ray propagations.")
    G,CmVar,mexact,x = generateModelFromOctree(octTree,len(rays),propName='Ne')
    
    #generate model
    print("Computing fake observations")
    dobs = []
    for i in range(20):
        dobs.append(G.dot(np.random.multivariate_normal(mean=mexact, cov = Cmexact)))
        #print(dobs[-1])
    dobs = np.array(dobs)
    Cd = np.cov(dobs.transpose())
    dobs = np.mean(dobs,axis=0)
    #print (dobs)
    mprior = mexact*0.5
    print("Solving for model from rays:")
    #m,Cm = LinearSolution(dobs,G,Cd,Cmexact,mprior)
    m,Cm = SteepestDescent(octTree,rays,dobs,Cd,Cmexact,mprior)
    CmCm = Cm.dot(np.linalg.inv(Cmexact))
    R = np.eye(CmCm.shape[0]) - CmCm
    print("Resolved by dataSet:{0}, resolved by a priori:{1}".format(np.trace(R),np.trace(CmCm)))
    import pylab as plt
    plt.plot(m,label='res')
    plt.plot(mexact,label='ex')
    plt.plot(mprior,label='pri')
    plt.legend(frameon=False)
    plt.show()
    plotOctTreeXZ(octTree,ax=None)
    plotOctTree3D(octTree,model=m)
   

Generating 3 component model
Computing model 3d exponential covariance
Setting octTree with model.
Generating 10 recievers
Generating 10 sources
Propagating 100 rays
Pulling ray propagations.
Computing fake observations


C:\Users\josh\WinPython-64bit-3.5.1.3\notebooks\RadioAstronomyThings\IonoTomo\IonoTomo\Geometry.py:113: RuntimeWarning: invalid value encountered in true_divide
  self.dir = np.array(direction)/np.sqrt(np.dot(direction,direction))


Solving for model from rays:
Setting octTree with model_0
Propagating 100 rays
Pulling ray propagations.
Doing forward problem
Calculating residuals, Sum:
32613216.5828
Calculating weighting residuals
Calculating adjoint
updating model
Setting octTree with model_1
Propagating 100 rays
Pulling ray propagations.
Doing forward problem
Calculating residuals, Sum:
31868206.7383
Calculating weighting residuals
Calculating adjoint
updating model
Setting octTree with model_2
Propagating 100 rays
Pulling ray propagations.
Doing forward problem
Calculating residuals, Sum:
31150301.936
Calculating weighting residuals
Calculating adjoint
updating model
Setting octTree with model_3
Propagating 100 rays
Pulling ray propagations.
Doing forward problem
Calculating residuals, Sum:
30458377.1144
Calculating weighting residuals
Calculating adjoint
updating model
Setting octTree with model_4
Propagating 100 rays
Pulling ray propagations.
Doing forward problem
Calculating residuals, Sum:
29791358.7443
Calc

In [13]:
f,(ax1,ax2) = plt.subplots(2,1,sharex=True,sharey=True)
ax1.imshow(Cm)
ax2.imshow(Cmexact)
plt.show()



In [20]:
m,Cm = LinearSolution(dobs,G,Cd,Cmexact,mprior)
CmCm = Cm.dot(np.linalg.inv(Cmexact))
R = np.eye(CmCm.shape[0]) - CmCm
print("Resolved by dataSet:{0}, resolved by a priori:{1}".format(np.trace(R),np.trace(CmCm)))
import pylab as plt
plt.plot(m,label='res')
plt.plot(mexact,label='ex')
plt.plot(mprior,label='pri')
plt.legend(frameon=False)
plt.show()

Doing forward problem
Calculating residuals:
[ 620.73379001  612.43555718  625.04747154  625.55605035  609.69680274
  618.18564228  647.67288382  636.20285667  627.2608913   627.2226498
  621.48061263  592.98669616  613.24361466  620.85809097  631.14864159
  614.8037199   619.63701376  625.77617391  626.25155257  598.69720273
  630.00192243  619.57852847  601.930617    617.68291439  616.09457505
  617.53253207  602.19013662  620.4429722   616.68904706  608.89894137
  634.87338052  613.2963879   610.98232291  639.11835163  635.81070611
  638.92145126  620.61570252  610.83829823  617.23463426  599.97924891
  631.5696202   598.21381942  617.8253217   603.80898283  649.27659824
  638.1479949   610.07843517  625.74916196  626.10866403  623.90838484
  648.34009341  603.13666391  608.8346001   598.03984401  633.43091185
  643.25921885  630.94527188  610.042684    614.44261171  632.29538341
  634.00662519  631.37903375  611.30040633  608.6322505   627.93160079
  624.71408711  637.92383139  624